In [40]:
#!pip -q install treelib

In [41]:
import pandas as pd;
import numpy as np;
import math;
import treelib as tl;
import ArvoreID3 as id3
import importlib
importlib.reload(id3)
import sys
from contextlib import redirect_stdout

In [42]:
basePlayTennis = pd.read_csv('../content/sample_data/play_tennis.csv', sep=',')
baseTitanic = pd.read_csv('../content/sample_data/train.csv', sep=',')

In [43]:
baseTitanicTratada = baseTitanic.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [44]:
# Para saber quantos valores nulos há em cada coluna de um dataframe pandas:
baseTitanicTratada.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [45]:

menor_valor = baseTitanicTratada['Age'].min()
maior_valor = baseTitanicTratada['Age'].max()
print(f"Menor valor: {menor_valor}, Maior valor: {maior_valor}")
menor_valor = baseTitanicTratada['Fare'].min()
maior_valor = baseTitanicTratada['Fare'].max()
print(f"Menor valor: {menor_valor}, Maior valor: {maior_valor}")

Menor valor: 0.42, Maior valor: 80.0
Menor valor: 0.0, Maior valor: 512.3292


In [46]:
baseTitanicTratada['Age'] = baseTitanicTratada['Age'].fillna(baseTitanicTratada['Age'].median())
baseTitanicTratada['Embarked'] = baseTitanicTratada['Embarked'].fillna(baseTitanicTratada['Embarked'].mode()[0])

In [47]:
faixas_etarias = [
    (0, 12, 'Crianças'),
    (12, 18, 'Adolescentes'),
    (18, 26, 'Jovens Adultos'),
    (26, 56, 'Adultos'),
    (56, np.inf, 'Idosos')
]

def categorizar_idade(idade):
    for faixa in faixas_etarias:
        if faixa[0] <= idade < faixa[1]:
            return faixa[2]
    print(f"Idade fora das faixas definidas: {idade}")
    return np.nan


# print(baseTitanicTratada.isnull().sum())

baseTitanicTratada['Age'] = baseTitanicTratada['Age'].apply(
    lambda x: categorizar_idade(float(str(x).split('-')[0])) if pd.notnull(x) else np.nan
)

In [48]:
quartis = baseTitanicTratada['Fare'].quantile([0.25, 0.5, 0.75])
def faixa_fare(valor):
    if valor <= quartis[0.25]:
        return 'Baixo'
    elif valor <= quartis[0.5]:
        return 'Média-Baixa'
    elif valor <= quartis[0.75]:
        return 'Média-Alta'
    else:
        return 'Alta'

baseTitanicTratada['Fare'] = baseTitanicTratada['Fare'].apply(faixa_fare)
baseTitanicTratada['Fare'].unique()

array(['Baixo', 'Alta', 'Média-Baixa', 'Média-Alta'], dtype=object)

In [49]:
baseTitanicTratada

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,Jovens Adultos,1,0,Baixo,S
1,1,1,female,Adultos,1,0,Alta,C
2,1,3,female,Adultos,0,0,Média-Baixa,S
3,1,1,female,Adultos,1,0,Alta,S
4,0,3,male,Adultos,0,0,Média-Baixa,S
...,...,...,...,...,...,...,...,...
886,0,2,male,Adultos,0,0,Média-Baixa,S
887,1,1,female,Jovens Adultos,0,0,Média-Alta,S
888,0,3,female,Adultos,1,2,Média-Alta,S
889,1,1,male,Adultos,0,0,Média-Alta,C


In [50]:
# Defina o nome do arquivo de saída
nome_arquivo = 'meu_resultado.txt'

# Use o 'with' para garantir que o arquivo seja fechado
with open(nome_arquivo, 'w') as f:
    # Redireciona a saída de tudo que estiver dentro do bloco 'with'
    with redirect_stdout(f):

        atributosGeraisTennis = basePlayTennis.drop(columns=['play', 'day'], axis=1)
        previsaoTennis = basePlayTennis[['play']]
        atributosGeraisTitanic = baseTitanicTratada.drop(columns=['Survived'], axis=1)
        previsaoTitanic = baseTitanicTratada[['Survived']]

        arvore = id3.ArvoreID3()
        #x_treino, y_treino, x_test, y_test = arvore.gerar_test_treino(atributosGerais=atributosGeraisTennis, atributosPrevisao=previsaoTennis, valorAleatorização=42, tamanhoTeste=0.4)
        x_treino, y_treino, x_test, y_test = arvore.gerar_test_treino(atributosGerais=atributosGeraisTitanic, atributosPrevisao=previsaoTitanic, valorAleatorização=42, tamanhoTeste=0.4)
        arvore.gerar_arvore_ID3(x_treino, y_treino)
        arvore.show()
        arvore.exibirRegras()
        matrizdeconfusao = arvore.gerar_metricas(x_test, y_test)
        print(matrizdeconfusao)